In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/llms/llms_fine_tune'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/llms/llms_fine_tune


In [ ]:
!pip install -q accelerate peft transformers trl bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install -q datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `write` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `write`


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

In [ ]:
# # Shuffle the dataset and slice it
# # dataset = dataset['train'].shuffle(seed=42).select(range(10000))
# train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)
# train_dataset = train_test_split['train']
# test_dataset = train_test_split['test']
# #
# # Define a function to transform the data
# def transform_conversation(example):

#     system_prompt = "System prompt: This is a medical question-answering session."
#     human_text = example['Question']
#     assistant_text = example['Answer']

#     # Format according to LLaMA 2's prompt template
#     reformatted_text = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n{human_text} [/INST] {assistant_text} </s>"

#     return {'text': reformatted_text}

# transformed_train_dataset = train_dataset.map(transform_conversation, remove_columns=["qtype", "Question", "Answer"])
# transformed_test_dataset = test_dataset.map(transform_conversation, remove_columns=["qtype", "Question", "Answer"])


# # Apply the transformation
# # transformed_dataset = dataset.map(transform_conversation)

In [ ]:
# transformed_train_dataset.push_to_hub("MedQuad-MedicalQnADataset_train")
# transformed_test_dataset.push_to_hub("MedQuad-MedicalQnADataset_test")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/missjd123/MedQuad-MedicalQnADataset_test/commit/18e5b150d243972c87903a1ce3a4d803e3d92f2d', commit_message='Upload dataset', commit_description='', oid='18e5b150d243972c87903a1ce3a4d803e3d92f2d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/missjd123/MedQuad-MedicalQnADataset_test', endpoint='https://huggingface.co', repo_type='dataset', repo_id='missjd123/MedQuad-MedicalQnADataset_test'), pr_revision=None, pr_num=None)

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-3.2-1B"

# The instruction dataset to use
dataset_name = "missjd123/MedQuad-MedicalQnADataset_train"

# Fine-tuned model name
new_model = "llama-3.2-1B-medical-qa"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# # Activate 4-bit precision base model loading
# use_4bit = False

# # Compute dtype for 4-bit base models
# bnb_4bit_compute_dtype = "float16"

# # Quantization type (fp4 or nf4)
# bnb_4bit_quant_type = "nf4"

# # Activate nested quantization for 4-bit base models (double quantization)
# use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/content/drive/My Drive/llms/llms_fine_tune/results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split='train')

# Load tokenizer and model with QLoRA configuration
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )

# Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)



README.md:   0%|          | 0.00/277 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13125 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

Step,Training Loss
25,1.978500
50,2.663300
75,1.902900
100,1.841300
125,1.542100
150,1.400100
175,1.482400
200,1.334100
225,1.265900
250,1.288800


TrainOutput(global_step=9846, training_loss=1.0631539210928376, metrics={'train_runtime': 4117.1412, 'train_samples_per_second': 9.564, 'train_steps_per_second': 2.391, 'total_flos': 7.024837749468365e+16, 'train_loss': 1.0631539210928376, 'epoch': 3.0})

In [ ]:
prompt = "What are the treatments for Lymphocytic Choriomeningitis (LCM) ?"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors='pt').to(device)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=1000,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id
    )

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What are the treatments for Lymphocytic Choriomeningitis (LCM)? - LCM Treatment - LCM Disease Information from Medindia
LCM is a viral infection of the nervous system. It causes no permanent damage to the nervous system but can cause symptoms that last from a few days to a few months. It can be caused by LCM virus, but it can also be caused by a number of other viruses. Symptoms may include headache, fever, weakness, irritability, drowsiness, confusion, loss of balance, and nausea. Some people may develop convulsions. In some people, the infection spreads to the spinal cord or brain and can cause permanent damage.
The LCM virus is spread through contact with infected animal products, such as raw meat, raw milk, unpasteurized milk, raw eggs, raw fish, and raw shellfish. It can also spread through contact with infected blood or body fluids, such as semen or saliva.
It is important to note that people with LCM usually recover without any treatment. However, some people may have long-term 

In [ ]:
model.push_to_hub("LLAMA-3.2-1B-medical-qa")

model.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hitmanonholiday/LLAMA-3.2-1B-medical-qa/commit/101173a9bf221e56ea00d11dfce69d1d32415aff', commit_message='Upload LlamaForCausalLM', commit_description='', oid='101173a9bf221e56ea00d11dfce69d1d32415aff', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hitmanonholiday/LLAMA-3.2-1B-medical-qa', endpoint='https://huggingface.co', repo_type='model', repo_id='hitmanonholiday/LLAMA-3.2-1B-medical-qa'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("LLAMA-3.2-1B-medical-qa")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hitmanonholiday/LLAMA-3.2-1B-medical-qa/commit/5e0cf8ee5438ef0964bc53a55d6c5eb7df822124', commit_message='Upload tokenizer', commit_description='', oid='5e0cf8ee5438ef0964bc53a55d6c5eb7df822124', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hitmanonholiday/LLAMA-3.2-1B-medical-qa', endpoint='https://huggingface.co', repo_type='model', repo_id='hitmanonholiday/LLAMA-3.2-1B-medical-qa'), pr_revision=None, pr_num=None)

In [ ]:
trainer.model.push_to_hub("LLAMA-3.2-1B-medical-qa")

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hitmanonholiday/LLAMA-3.2-1B-medical-qa/commit/b078d3360ef3bed99bc277517c87bfbc71600524', commit_message='Upload model', commit_description='', oid='b078d3360ef3bed99bc277517c87bfbc71600524', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hitmanonholiday/LLAMA-3.2-1B-medical-qa', endpoint='https://huggingface.co', repo_type='model', repo_id='hitmanonholiday/LLAMA-3.2-1B-medical-qa'), pr_revision=None, pr_num=None)

In [ ]:
%cd /content/drive/My Drive/llms/llms_fine_tune/

/content/drive/My Drive/llms/llms_fine_tune


In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Is D-bifunctional protein deficiency inherited ?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] Is D-bifunctional protein deficiency inherited ? [/INST] D-bifunctional protein deficiency is inherited in an autosomal recessive pattern, which means both copies of the gene in each cell have mutations. everybody has two copies of each gene, one inherited from each parent. When a person has a mutation in one copy of the gene and no mutation in the other copy, they are called a carrier.
